In [114]:
from dotenv import load_dotenv
from openai import OpenAI
import openai
import os
openai_client = OpenAI()

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [115]:
def LLMResponse(prompt):
    # Chat Text Template
    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant knowledgeable in AI/ML concepts."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return completion

In [116]:
# prompt = "Suggest an AI/ML topic that can be explained in under 150 words. Please provide only the topic without a description."
# topic_response = chat_completion(prompt)


# Chat Text Template
# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant knowledgeable in AI/ML concepts."},
#         {
#             "role": "user",
#             "content": "Suggest an AI/ML topic that can be explained in under 150 words. Please provide only the topic without a description."
#         }
#     ]
# )

In [117]:
# topic = topic_response.choices[0].message.content

In [118]:

# completion_content = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a knowledgeable assistant in AI/ML concepts."},
#         {
#             "role":"user",
#             "content":f"Explain the concept of {topic} in simple terms in less than 150 words."
#         }
#     ]
# )

In [119]:
# content = completion_content.choices[0].message.content

In [120]:
topic_history = set()

def generate_content():
    global topic_history
    max_attempts = 5
    
    for _ in range(max_attempts):
        # get the current topic
        topic_response = LLMResponse(f"Suggest a unique AI/ML topic that can be explained in under 150 words. Please provide only the topic without a description. Avoid these topics: {', '.join(topic_history)}.")
        topic = topic_response.choices[0].message.content.strip().lower()
        
        if topic not in topic_history:
            topic_history.add(topic)
            # get the current content
            content_response = LLMResponse(f"Explain the concept of {topic} in simple terms in less than 150 words.")
            content = content_response.choices[0].message.content
            return topic, content
    
    raise Exception("Unable to generate a unique topic after multiple attempts")

In [121]:
from pymongo import MongoClient

# MongoDB connection settings
MONGO_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "reels-content"
COLLECTION_NAME = "content"

# Connect to MongoDB
mongo_client = MongoClient(MONGO_URI)
db = mongo_client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

try:
    topic, content = generate_content()
except Exception as e:
    print(f"Error generating content: {e}")
    
# # retrieving all the documents from the collection
# documents = collection.find()

# # checking if the current topic already exists in any of the documents to avoid redundancy
# for document in documents:
#     if topic in document:
#         topic, content = generate_content()

output_dict = {topic: content}

# Insert dictionary into MongoDB
result = collection.insert_one(output_dict)

# Close MongoDB connection
mongo_client.close()

print("Data inserted with ID:", result.inserted_id)

Data inserted with ID: 66ee30369103e9572dbc8d6b
